In [1]:
%matplotlib inline

# Keras 全相连网络(Full Connected Network)

In [2]:
# -*- coding: utf-8 -*-

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from six.moves import xrange

导入MNIST数据集

In [3]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('训练集大小: x =', x_train.shape, ', y =', y_train.shape)
print('测试集大小: x =', x_test.shape, ', y =', y_test.shape)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


训练集大小: x = (60000, 28, 28) , y = (60000,)
测试集大小: x = (10000, 28, 28) , y = (10000,)


数据展示

In [4]:
from PIL import Image

Image.fromarray(x_train[0])

In [5]:
y_train[0]

5

reshape/flatten: image from 2D (28*28) to 1D (784)

In [6]:
import keras

# hyper-parameter
training_epoch=50
num_classes = 10
learning_rate=1e-3

x_train = x_train.reshape(x_train.shape[0], -1).astype('float32') / 255
x_test = x_test.reshape(x_test.shape[0], -1).astype('float32') / 255

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print('training data: x =', x_train.shape, ', y =', y_train.shape)
print('test data: x =', x_test.shape, ', y =', y_test.shape)

training data: x = (60000, 784) , y = (60000, 10)
test data: x = (10000, 784) , y = (10000, 10)


one-hot encoding

In [7]:
x_train[0].shape

(784,)

In [8]:
y_train[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

## 正则化实现(regularization)

### 正则化(l1, l2, l1_l2)
``
from keras import regularizers
from keras.layers import Dense
``<br>
``
Dense(kernel_regularizer=regularizers.l2(1e-3),
      bias_regularizer=regularizers.l2(1e-3))
``

### 自定义正则化
``
from keras import backend as K
def l1_reg(weight_matrix):
    return 0.01 * K.sum(K.abs(weight_matrix))
Dense(64, input_dim=64, kernel_regularizer=l1_reg)
``

## Batch Norm
``
from keras.layers import BatchNormalization
BatchNormalization()(input_tensor)
``

In [9]:
image_size = x_train.shape[-1]

from keras.models import Sequential
from keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(image_size,)))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               65664     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 468,874
Trainable params: 468,874
Non-trainable params: 0
_________________________________________________________________


In [10]:
from keras.optimizers import *

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate),
              metrics=['accuracy'])

训连 training

`model.fit`函数接口
```
fit(self, x, y, batch_size=32, epochs=10, verbose=1, callbacks=None,
            validation_split=0., validation_data=None, shuffle=True,
            class_weight=None, sample_weight=None, initial_epoch=0, **kwargs)
```
其中`x,y`表示全部数据集；callbacks是高级控制接口；`verbose`为1表示输出每一次batch的训练结果，为0表示不做任何输出；`validation_split`在0~1，是将训练集按照一定比例划分为验证集；`validation_data`是验证集，训练时加入验证集有助有模型的训练过程，会使得模型着重学习验证集，可能会导致模型性能突发提升；`initial_epoch`是模型的出事训练周期数，这对于一些自适应优化器和加载模型参数非常重要。

In [11]:
model.fit(x_train, y_train,
          batch_size=1000,
          epochs=training_epoch,
          verbose=1)

Epoch 1/50
60000/60000 [==============================] - 1s 23us/step - loss: 0.6194 - acc: 0.8158
Epoch 2/50
60000/60000 [==============================] - 0s 8us/step - loss: 0.2164 - acc: 0.9361
Epoch 3/50
60000/60000 [==============================] - 0s 8us/step - loss: 0.1537 - acc: 0.9551
Epoch 4/50
60000/60000 [==============================] - 0s 7us/step - loss: 0.1189 - acc: 0.9654
Epoch 5/50
60000/60000 [==============================] - 0s 8us/step - loss: 0.0969 - acc: 0.9714
Epoch 6/50
60000/60000 [==============================] - 0s 8us/step - loss: 0.0806 - acc: 0.9760
Epoch 7/50
60000/60000 [==============================] - 0s 7us/step - loss: 0.0687 - acc: 0.9795
Epoch 8/50
60000/60000 [==============================] - 0s 7us/step - loss: 0.0592 - acc: 0.9819
Epoch 9/50
60000/60000 [==============================] - 0s 7us/step - loss: 0.0509 - acc: 0.9843
Epoch 10/50
60000/60000 [==============================] - 0s 8us/step - loss: 0.0467 - acc: 0.9855
Epoch 11

检验 estimaton

In [12]:
loss, acc = model.evaluate(x_test, y_test, verbose=0)
print('test loss: %s, test accuracy: %s' % (loss, acc))

test loss: 0.07331758836482286, test accuracy: 0.983


In [13]:
test = x_test[0]
Image.fromarray(test.reshape(28, 28), 'I')

In [14]:
import numpy as np

test = test[None, :]
# 模型预测
x_predicted = model.predict(test)
np.argmax(x_predicted)

7